In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.cosmology import WMAP9 as cosmo
import astropy.constants as const

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Times"],
    "text.latex.preamble": r"\usepackage{amsmath}\usepackage{mathptmx}",  # Times Roman
    "hatch.linewidth": 3.0,
})
sns.set_context("paper",font_scale=1)


def separate_mag(col):
    m = [list(map(float,a.split("/"))) for a in col]
    bigM = np.array([np.max(m) for m in m])
    smallM = np.array([np.min(m) for m in m])
    return bigM, smallM

# calculate luminosity
def cal_lum(mag,wl,redshifts):
    f = 10**((mag+21.1)/-2.5) *u.erg/u.s/u.cm**2/u.angstrom
    lum = (4*np.pi *cosmo.angular_diameter_distance(redshifts)**2 *f *(wl*u.angstrom)).to(u.erg/u.s)
    return lum


def hist_bin(ax,quant,lab,clr):
    """plot histogram and a straight line at peak bin"""
    try:
        count, bin, _ = ax.hist(quant,bins=np.linspace(quant.min(),quant.max(),int(np.sqrt(len(quant)))),
                histtype='step',color=clr,alpha=0.6)
        maxind = np.argmax(count)
        maxloc = (bin[maxind]+bin[maxind+1])/2
        ax.axvline(maxloc,c=clr,label=lab)
    except:
        ax.axvline(np.mean(quant),c=clr,label=lab)
    
def meas_df(rp,quartiles,quant,ax,dualmask,cuts):
    """plot sep histograms of luminosity bins based on list of quartiles"""
    if cuts == []:
        cuts =  np.percentile(quant,quartiles)
    colors = sns.color_palette("magma", len(cuts)+1)
    for i,c in zip(range(len(cuts)+1),colors):
        if i==0:
            lbl = f'$<$ {cuts[i]:.2f}'
            hist_bin(ax[0],rp[(quant < cuts[i]) & dualmask], lbl, c)
        elif i==len(cuts):
            lbl = f'$>$ {cuts[i-1]:.2f}'
            hist_bin(ax[0],rp[(quant > cuts[i-1]) & dualmask], lbl, c)
        else:
            lbl = f'{cuts[i-1]:.2f} $-$ {cuts[i]:.2f}'
            hist_bin(ax[0],rp[(quant < cuts[i]) & (quant > cuts[i-1]) & dualmask], lbl, c)
    ax[0].legend(fontsize=8,loc='upper left')
    ax[1].hist(quant[dualmask],histtype='step')
    [ax[1].axvline(i) for i in cuts]
    ax[0].set_xlabel("Sep [kpc]")
    ax[0].set_ylabel("Number of dual")
    ax[1].set_xlabel("Log(Luminosity) [erg/s]")
    ax[1].set_ylabel("Number of dual")
    ;



### varstro data

In [ ]:
with open("/home/insepien/research-data/df/varsFR.txt", "r") as f:
    vars = [a.rstrip().split("\t") for a in f.readlines()]

vardf = pd.DataFrame(vars[2:],columns=vars[0]) 

sep = vardf['r _p'].values.astype(float)

# separate magnitudes of main and secondary
big_m475, small_m475 = separate_mag(vardf['F475W'])

redshifts = np.array([float(z.split("^")[0]) if z[-1]=="r" else float(z) for z in vardf['Redshift']])
lum475b = cal_lum(big_m475,4750)
lum475a = cal_lum(small_m475,4750)

# # checking the first object luminosity with WISE, close?
# obs_flux = 3.80E-2*u.Jy
# obs_hz = 1.36E+13*u.Hz
# lum = (obs_flux*obs_hz*4*np.pi*
#         cosmo.luminosity_distance(0.7)**2).to(u.erg/u.s)
# lum*10, (lum475a+lum475b)[0]*1000
vdf = pd.DataFrame([redshifts,sep,vardf['Classification']],index=['z','sep','type']).T

In [ ]:
fig,ax = plt.subplots(3,2,gridspec_kw={'height_ratios': [1,1,1.5]},figsize=(8,7))#,dpi=200)

# masks for varstro paper
b =  np.array([a[-2:]=="^b" for a in vardf['Classification']]) # chen22, star based on gemini spectra
c = np.array([a[-2:]=="^c" for a in vardf['Classification']]) # chen22, star based on sdss spectra
chen24star = vardf['Classification'].str.contains("star chen24") # chen 24
gross24single =  vardf['Classification'].str.contains("gross24") # chen 24
zmax = redshifts.max()
zmin = 0.5
zmask = (redshifts> zmin) & (redshifts<zmax)
dualmask = vardf['Classification'].str.contains("dual")
#dualmask = ~chen24star & ~gross24single & zmask
#dualmask = ~ vardf['Classification'].str.contains("star")
#dualmask = np.array([True]*len(vdf))

quant = np.log10((lum475b).value)
cuts0 = []
cuts1 = [43,43.5]
meas_df(sep,[50],quant,ax[0,:],dualmask,cuts0)
meas_df(sep,[25,75],quant,ax[1,:],dualmask,cuts1)


zarr = np.linspace(zmin,zmax)
hstRes = ((0.04*3*u.arcsec).to(u.rad).value*cosmo.angular_diameter_distance(zarr)).to(u.kpc).value
ax[-1,0].plot(zarr,hstRes,alpha=0.5,label="0.12''")
gaiaRes = ((0.4*u.arcsec).to(u.rad).value*cosmo.angular_diameter_distance(zarr)).to(u.kpc).value
ax[-1,0].plot(zarr,gaiaRes,alpha=0.5,label="0.4''")
gaiaResUp = ((0.7*u.arcsec).to(u.rad).value*cosmo.angular_diameter_distance(zarr)).to(u.kpc).value
ax[-1,0].plot(zarr,gaiaResUp,alpha=0.5,label="0.7''")


sca = ax[-1,0].scatter(redshifts[dualmask],sep[dualmask], cmap="magma",c=quant[dualmask])
cax = fig.add_axes([0.5, 0.125, 0.03, 0.25])
cbar = fig.colorbar(sca,cax=cax)
cbar.set_label("Log(L$_{\\rm{457 W, total}}$) [erg/s]")
ax[-1,0].set_xlabel("Redshift")
ax[-1,0].set_ylabel("Sep(kpc)")
ax[-1,0].legend();
ax[-1,-1].axis("off")

fig.suptitle("By secondary AGN lum")
fig.tight_layout();


### liu11

In [ ]:
with open("/home/insepien/research-data/df/liu11.txt", "r") as f:
    liu = [a.rstrip().replace("   "," ").replace("  ", " ").split(" ") for a in f.readlines()[37:]]
liudf = pd.DataFrame(liu[1:],columns=liu[0]) 

sep = liudf['rp'].values.astype(float)
mag = liudf['rmag'].values.astype(float)
redshifts = liudf['z'].astype(float).values



In [ ]:
# need to check if mag provided is AB or asinh, but lum change shouldn't be too large
flx = 10**((22.5-mag)/2.5)*3.631e-6*u.Jy
nu = const.c/6231/u.angstrom
lum = (4*np.pi*(flx*nu)*cosmo.angular_diameter_distance(redshifts)**2).to(u.erg/u.s)

In [ ]:
fig,ax = plt.subplots(3,2,gridspec_kw={'height_ratios': [1,1,1.5]},figsize=(8,7))#,dpi=200)
quant = np.log10(lum.value)

dualmask = np.array(len(lum)*[True])#sep<10

cuts0 = []
cuts1 = []
meas_df(sep,[50],quant,ax[0,:],dualmask,cuts0)
meas_df(sep,[25,75],quant,ax[1,:],dualmask,cuts1)


sca = ax[-1,0].scatter(redshifts[dualmask],sep[dualmask], cmap="magma",c=quant[dualmask])
cax = fig.add_axes([0.5, 0.125, 0.03, 0.25])
cbar = fig.colorbar(sca,cax=cax)
cbar.set_label("Log(L$_{\\rm{r band, total}}$) [erg/s]")
ax[-1,0].set_xlabel("Redshift")
ax[-1,0].set_ylabel("Sep(kpc)")
ax[-1,0].legend();
ax[-1,-1].axis("off")

fig.suptitle("By total AGN lum")
fig.tight_layout();


### observed sep hist

In [ ]:
bigmac = pd.read_csv("/home/insepien/research-data/GFG.csv")
# create masks
positive_sep = bigmac['Sep(kpc)'] > 0 
dual_sep = (bigmac['Sep(kpc)']>0.1 ) #& (bigmac['Sep(kpc)']<20)
confd = bigmac['ST1 Confidence Flag']>=0
radio = bigmac['Parsed Analysis Method'].str.contains("Radio")
imaging = bigmac['Parsed Analysis Method'].str.contains("Imaging")
# get papers with largest sample
pp = []
for i in bigmac['Paper(s)']:
    [pp.append(j) for j in i.split(" ; ")]
paper_dict = Counter(pp).most_common()

In [ ]:
def norm_hist(ax,quant,fcolor,ecolor,bin_arr,lbl):
    """normalize histogram sum count to 1 given some quantity (quant)
        args: edgecoloe, facecolor, bin array, opacity, flag for plotting horizontal hist"""
    count, bin = np.histogram(quant,bins=bin_arr)
    ax.bar(bin[:-1],count/np.sum(count),width = np.diff(bin),align='edge',facecolor=fcolor,edgecolor=ecolor,label=lbl)
    #ax.plot(bin[:-1],count/np.sum(count),c=ecolor)
    
def plot_sep_hist(npaper, upsep):
    fig,ax = plt.subplots(npaper//5,5,figsize=(12,3*npaper//5),sharex=True, sharey=True)
    ax = ax.ravel()
    upsepvar = np.max(vard)
    numbin = int(np.sqrt(len(vard)))
    bins_vard = np.linspace(np.min(vard),upsepvar,numbin)
    bins = np.diff(bins_vard)[0]*np.arange(upsep)+1
    cls = sns.color_palette("colorblind", 10)
    small_sep = bigmac['Sep(kpc)']< upsep
    for p,a in zip(paper_dict[:npaper+1],ax):
        mask = [np.isin(p[0],i.split(" ; ")) == True for i in bigmac['Paper(s)']]
        #plt.hist(bigmac[mask&positive_sep]['Sep(kpc)'],histtype="step",label=p[0])
        norm_hist(a,vard,"darkseagreen","darkseagreen",bins,'varstro')
        norm_hist(a,bigmac[mask & positive_sep & small_sep]['Sep(kpc)'],"none","blue",bins,p[0])
        a.legend()
    fig.tight_layout();
